In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [2]:
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
import pandas as pd
import numpy as np
import chart_studio.plotly as py 
from datetime import timedelta
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from pandas_profiling import ProfileReport
pio.templates.default = 'plotly_dark'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_map = {'high': '#ef553b', 'low': '#636efa'}
import datetime
import pytz

In [3]:
querySno = SQLQuery('snowflake')
pilot_df = querySno(r"""
with cashins as (
select product_id, business_id,
max(case when rnk = 1 then transaction_id else null end) as first_cashin_trans_id,
max(case when rnk = 2 then transaction_id else null end) as second_cashin_trans_id,
max(case when rnk = 3 then transaction_id else null end) as third_cashin_trans_id,
max(case when rnk = 1 then a.created_at else null end) as first_cashin_date,
max(case when rnk = 2 then a.created_at else null end) as second_cashin_date,
max(case when rnk = 3 then a.created_at else null end) as third_cashin_date
--min(transaction_date) as first_squareoff_date

from (
select * from 
(select *, row_number() over(partition by product_id order by created_at) rnk
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LOAN_PRODUCT_TRANSACTIONS
where action_type='CASHIN' 
and (method = 'AUTOMATED' or description ilike 'Cashin done by admin%%')
--and created_at < to_date('2022-12-01')
)
where rnk<4
) a 
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LOAN_PRODUCTS b on a.product_id = b.id

group by 1,2
),

first_squareoff as(
select product_id,transaction_date as first_squareoff_date from (
select a.*,
f.transaction_date, row_number() over(partition by a.business_id order by f.transaction_date) rnk
from cashins a
left join prod_db.data.transactions f on a.business_id = f.business_id
where date(f.transaction_date) >= date(a.first_cashin_date)
and f.running_balance >= 0
)
where rnk = 1
),

second_squareoff as(
select product_id,transaction_date as second_squareoff_date from (
select a.*,
s.transaction_date, row_number() over(partition by a.business_id order by s.transaction_date) rnk
from cashins a
left join prod_db.data.transactions s on a.business_id = s.business_id
where date(s.transaction_date) >= date(a.second_cashin_date)
and s.running_balance >= 0
)
where rnk = 1
),

third_squareoff as(
select product_id,transaction_date as third_squareoff_date from (
select a.*,
t.transaction_date, row_number() over(partition by a.business_id order by t.transaction_date) rnk
from cashins a
left join prod_db.data.transactions t on a.business_id = t.business_id
where date(t.transaction_date) >= date(a.third_cashin_date)
and t.running_balance >= 0
)
where rnk = 1
)

select a.product_id, a.business_id,first_cashin_date as pilot_first_due_date, second_cashin_date as pilot_second_due_date,third_cashin_date as pilot_third_due_date,
case when f.running_balance < 0 then 1 else 0 end as pilot_fpd,
case when s.running_balance < 0 then 1 else 0 end as pilot_spd,
case when t.running_balance < 0 then 1 else 0 end as pilot_tpd,
datediff(day,a.first_cashin_date,fs.first_squareoff_date) as pilot_first_cure_days,
datediff(day,a.second_cashin_date,ss.second_squareoff_date) as pilot_second_cure_days,
datediff(day,a.third_cashin_date,ts.third_squareoff_date) as pilot_third_cure_days

from "PROD_DB"."ADHOC"."LENDING_PILOT" lp
join cashins a on lp.business_id = a.business_id
left join first_squareoff fs on a.product_id = fs.product_id
left join second_squareoff ss on a.product_id = ss.product_id
left join third_squareoff ts on a.product_id = ts.product_id

left join prod_db.data.transactions f
on a.first_cashin_trans_id = f.transaction_id
left join prod_db.data.transactions s 
on a.second_cashin_trans_id = s.transaction_id
left join prod_db.data.transactions t 
on a.third_cashin_trans_id = t.transaction_id

where third_cashin_date is not null
"""
)

In [4]:
pilot_df.shape

(643, 11)

In [5]:
pilot_df.head()

,product_id,business_id,pilot_first_due_date,pilot_second_due_date,pilot_third_due_date,pilot_fpd,pilot_spd,pilot_tpd,pilot_first_cure_days,pilot_second_cure_days,pilot_third_cure_days
0,e2feec25-251b-45a8-9f37-0304a0f30862,fdc7154e-18b3-4c26-b3b1-5c748f2facdb,2022-09-06 07:00:11.333000+00:00,2022-10-07 07:00:24.186000+00:00,2022-11-07 05:51:20.582000+00:00,0,0,1,0,0,60
1,28910777-199c-48d9-8a1e-a846e3040b08,35308ad5-7271-4703-a362-18a4d79cd334,2022-09-11 07:00:11.946000+00:00,2022-10-11 07:00:13.569000+00:00,2022-10-11 07:00:15.332000+00:00,0,0,0,0,0,0
2,bb977a66-20bf-4461-833b-046e67d88ef9,5cbc0f97-efb1-470a-9952-52a30b053386,2022-09-07 07:00:03.070000+00:00,2022-11-27 08:00:31.660000+00:00,2022-12-27 08:00:07.952000+00:00,1,0,0,6,0,0
3,eee7de77-1646-48b5-a9b6-d469d0a8a208,623d4577-8472-42c5-809b-1d72d8625b8b,2022-08-22 05:00:11.972000+00:00,2022-09-12 07:00:07.479000+00:00,2022-09-21 07:00:14.824000+00:00,1,0,0,0,0,0
4,85a98d0f-5fb3-4dcd-9e47-435cf8555784,ab59e1a9-605b-43b4-8312-880b4123bc28,2022-10-01 07:00:44.111000+00:00,2022-10-15 07:06:47.568000+00:00,2022-10-31 07:00:16.200000+00:00,0,0,1,0,0,0


In [6]:
pilot_df = pilot_df[~pilot_df['pilot_third_due_date'].isnull()]

In [7]:
pilot_df.shape

(643, 11)

In [8]:
pilot_df.columns

Index(['product_id', 'business_id', 'pilot_first_due_date',
       'pilot_second_due_date', 'pilot_third_due_date', 'pilot_fpd',
       'pilot_spd', 'pilot_tpd', 'pilot_first_cure_days',
       'pilot_second_cure_days', 'pilot_third_cure_days'],
      dtype='object')

In [9]:
pilot_df['FPD_plus_3'] = 0
pilot_df['SPD_plus_3'] = 0
pilot_df['TPD_plus_3'] = 0
pilot_df.loc[(pilot_df['pilot_fpd']==1) & (pilot_df['pilot_first_cure_days']>=3),'FPD_plus_3'] = 1
pilot_df.loc[(pilot_df['pilot_spd']==1) & (pilot_df['pilot_second_cure_days']>=3),'SPD_plus_3'] = 1
pilot_df.loc[(pilot_df['pilot_tpd']==1) & (pilot_df['pilot_third_cure_days']>=3),'TPD_plus_3'] = 1

In [10]:
pilot_df['FPD_plus_15'] = 0
pilot_df['SPD_plus_15'] = 0
pilot_df['TPD_plus_15'] = 0
pilot_df.loc[(pilot_df['pilot_fpd']==1) & (pilot_df['pilot_first_cure_days']>=15),'FPD_plus_15'] = 1
pilot_df.loc[(pilot_df['pilot_spd']==1) & (pilot_df['pilot_second_cure_days']>=15),'SPD_plus_15'] = 1
pilot_df.loc[(pilot_df['pilot_tpd']==1) & (pilot_df['pilot_third_cure_days']>=15),'TPD_plus_15'] = 1

In [11]:
pilot_df['FPD_plus_30'] = 0
pilot_df['SPD_plus_30'] = 0
pilot_df['TPD_plus_30'] = 0
pilot_df.loc[(pilot_df['pilot_fpd']==1) & (pilot_df['pilot_first_cure_days']>=30),'FPD_plus_30'] = 1
pilot_df.loc[(pilot_df['pilot_spd']==1) & (pilot_df['pilot_second_cure_days']>=30),'SPD_plus_30'] = 1
pilot_df.loc[(pilot_df['pilot_tpd']==1) & (pilot_df['pilot_third_cure_days']>=30),'TPD_plus_30'] = 1

In [12]:
pilot_df.columns

Index(['product_id', 'business_id', 'pilot_first_due_date',
       'pilot_second_due_date', 'pilot_third_due_date', 'pilot_fpd',
       'pilot_spd', 'pilot_tpd', 'pilot_first_cure_days',
       'pilot_second_cure_days', 'pilot_third_cure_days', 'FPD_plus_3',
       'SPD_plus_3', 'TPD_plus_3', 'FPD_plus_15', 'SPD_plus_15', 'TPD_plus_15',
       'FPD_plus_30', 'SPD_plus_30', 'TPD_plus_30'],
      dtype='object')

In [18]:
querySno = SQLQuery('snowflake')
ga_df = querySno(r"""
with default_flags as (
select distinct lending_business_id, business_id,loan_month,
max(case when emi_no = 1 then due_date end) over(partition by lending_business_id) as first_duedate, 
max(case when emi_no = 2 then due_date end) over(partition by lending_business_id) as second_duedate, 
max(case when emi_no = 3 then due_date end) over(partition by lending_business_id) as third_duedate,
max(case when emi_no = 4 then due_date end) over(partition by lending_business_id) as fourth_duedate,
max(case when emi_no = 1 then defalt end) over(partition by lending_business_id) as fpd, 
max(case when emi_no = 2 then defalt end) over(partition by lending_business_id) as spd, 
max(case when emi_no = 3 then defalt end) over(partition by lending_business_id) as tpd,
max(case when emi_no = 4 then defalt end) over(partition by lending_business_id) as frpd,
max(case when emi_no = 1 then total_emi end) over(partition by lending_business_id) as first_emi_amount,
max(case when emi_no = 2 then total_emi end) over(partition by lending_business_id) as second_emi_amount,
max(case when emi_no = 3 then total_emi end) over(partition by lending_business_id) as third_emi_amount,
max(case when emi_no = 4 then total_emi end) over(partition by lending_business_id) as fourth_emi_amount
from (
select a.*,json_extract_path_text(c.predict_meta, 'business_id') as business_id, month(a.created_at) as month_no,
case when paid_on_time = 'true' then 0
when paid_on_time = 'false' then 1 else null end as default_flag,
dense_rank() over(partition by a.lending_business_id order by due_date) as emi_no,
min(a.created_at) over (partition by a.lending_business_id) as min_created_at,
month(min_created_at) as loan_month,
max(default_flag) over(partition by a.lending_business_id, due_date) as defalt,
sum(due_amount) over(partition by a.lending_business_id, due_date) as amount_due,
sum(payment) over(partition by a.lending_business_id, due_date) as payment_done,
amount_due/100 + payment_done/100 as total_emi
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS c on a.lending_business_id = c.lending_business_id
where date(due_date) < date('2023-05-20')
-- and a.lending_business_id = '909564ef-6b22-4303-9df5-fc4e892fef01'
)
)
, first_cure as (
select *,transaction_date as first_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) rnk
from (
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as first_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where first_amount_cured >= first_emi_amount
)
where rnk = 1
)
, second_cure as (
select *, transaction_date as second_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
t.running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as second_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where second_amount_cured >= first_emi_amount + second_emi_amount
)
where rnk = 1
)

,third_cure as (
select *, transaction_date as third_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as third_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where third_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount
)
where rnk = 1
)

,fourth_cure as (
select *, transaction_date as fourth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount ,t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as fourth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where fourth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount
)
where rnk = 1
)

select a.lending_business_id, a.business_id, a.first_duedate,date(f.first_cure_date)as first_cure_date, a.second_duedate,date(s.second_cure_date)as second_cure_date, 
a.third_duedate,date(t.third_cure_date) as third_cure_date, a.fourth_duedate,date(fr.fourth_cure_date) as fourth_cure_date,fpd, spd, tpd, frpd,
case when fpd = 1 and first_amount_cured >= a.first_emi_amount then 1 else 0 end as first_cure_flag,
case when spd = 1 and second_amount_cured >= a.first_emi_amount + a.second_emi_amount then 1 else 0 end as second_cure_flag,
case when tpd = 1 and third_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount then 1 else 0 end as third_cure_flag,
case when frpd = 1 and fourth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount then 1 else 0 end as fourth_cure_flag,
case when first_cure_flag = 1 then datediff(day,a.first_duedate,f.first_cure_date) else null end as first_cure_days,
case when second_cure_flag = 1 then datediff(day,a.second_duedate,s.second_cure_date) else null end as second_cure_days,
case when third_cure_flag = 1 then datediff(day,a.third_duedate,t.third_cure_date) else null end as third_cure_days,
case when fourth_cure_flag = 1 then datediff(day,a.fourth_duedate,fr.fourth_cure_date) else null end as fourth_cure_days,
a.first_emi_amount, f.first_amount_cured, a.second_emi_amount,(s.second_amount_cured - f.first_amount_cured) as second_amt_cured, a.third_emi_amount, 
(t.third_amount_cured - second_amt_cured) as third_amt_cured, a.fourth_emi_amount, (fr.fourth_amount_cured - third_amt_cured) as fourth_amt_cured
from default_flags a 
left join first_cure f on a.lending_business_id = f.lending_business_id
left join second_cure s on a.lending_business_id = s.lending_business_id
left join third_cure t on a.lending_business_id = t.lending_business_id
left join fourth_cure fr on a.lending_business_id = fr.lending_business_id
where a.first_duedate < date('2023-05-30')
-- and a.business_id = '111eb19b-cd06-4943-b9f5-c111ae38189e'
order by first_duedate desc
""")

In [19]:
ga_df.shape

(2509, 30)

In [20]:
ga_df.head()

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fourth_duedate,fourth_cure_date,fpd,spd,tpd,frpd,first_cure_flag,second_cure_flag,third_cure_flag,fourth_cure_flag,first_cure_days,second_cure_days,third_cure_days,fourth_cure_days,first_emi_amount,first_amount_cured,second_emi_amount,second_amt_cured,third_emi_amount,third_amt_cured,fourth_emi_amount,fourth_amt_cured
0,ba89d753-b8a9-42ff-9a9f-6a720ef39d21,9c8d5541-61ef-4203-ad12-d0a79722f4f2,2023-05-19,None,None,None,None,None,None,None,1,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,3882.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6801309f-40d6-4d9f-9a11-05a867e5ce2b,20f98713-f8e1-4a53-849e-4b2fcab7b0ab,2023-05-19,2023-05-19,None,None,None,None,None,None,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,1090.00,1090.0,NaN,NaN,NaN,NaN,NaN,NaN
2,489d5197-cc08-4e08-ab8d-e49fb3a2238d,None,2023-05-19,None,None,None,None,None,None,None,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,1008.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6621da35-034e-4949-95ca-b643e05b7995,None,2023-05-19,None,None,None,None,None,None,None,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,448.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,df213af7-be7f-4005-a560-71e02b14b0fc,None,2023-05-19,None,None,None,None,None,None,None,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,230.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
ga_df['FPD_plus_3'] = 0
ga_df['SPD_plus_3'] = 0
ga_df['TPD_plus_3'] = 0
ga_df['FRPD_plus_3'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=3),'FPD_plus_3'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=3),'SPD_plus_3'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=3),'TPD_plus_3'] = 1
ga_df.loc[(ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=3),'FRPD_plus_3'] = 1

In [22]:
ga_df['FPD_plus_15'] = 0
ga_df['SPD_plus_15'] = 0
ga_df['TPD_plus_15'] = 0
ga_df['FRPD_plus_15'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=15),'FPD_plus_15'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=15),'SPD_plus_15'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=15),'TPD_plus_15'] = 1
ga_df.loc[(ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=15),'FRPD_plus_15'] = 1

In [23]:
ga_df['FPD_plus_30'] = 0
ga_df['SPD_plus_30'] = 0
ga_df['TPD_plus_30'] = 0
ga_df['FRPD_plus_30'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=30),'FPD_plus_30'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=30),'SPD_plus_30'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=30),'TPD_plus_30'] = 1
ga_df.loc[(ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=30),'FRPD_plus_30'] = 1

#### GA Stacking

In [24]:
ga_first = ga_df[['business_id','first_duedate','FPD_plus_3','FPD_plus_15','FPD_plus_30']].rename(columns={'first_duedate': 'due_date', 'FPD_plus_3': 'DPD_plus_3','FPD_plus_15': 'DPD_plus_15','FPD_plus_30': 'DPD_plus_30'})
ga_second = ga_df[['business_id','second_duedate','SPD_plus_3','SPD_plus_15','SPD_plus_30']].rename(columns={'second_duedate': 'due_date', 'SPD_plus_3': 'DPD_plus_3','SPD_plus_15': 'DPD_plus_15','SPD_plus_30': 'DPD_plus_30'})
ga_third = ga_df[['business_id','third_duedate','TPD_plus_3','TPD_plus_15','TPD_plus_30']].rename(columns={'third_duedate': 'due_date', 'TPD_plus_3': 'DPD_plus_3','TPD_plus_15': 'DPD_plus_15','TPD_plus_30': 'DPD_plus_30'})
ga_fourth = ga_df[['business_id','fourth_duedate','FRPD_plus_3','FRPD_plus_15','FRPD_plus_30']].rename(columns={'fourth_duedate': 'due_date', 'FRPD_plus_3': 'DPD_plus_3','FRPD_plus_15': 'DPD_plus_15','FRPD_plus_30': 'DPD_plus_30'})

In [25]:
ga_first['emi_no'] = 1
ga_second['emi_no'] = 2
ga_third['emi_no'] = 3
ga_fourth['emi_no'] = 4

In [26]:
ga_stack = pd.concat([ga_first, ga_second, ga_third, ga_fourth], axis = 0, ignore_index = True)

In [27]:
ga_stack.shape

(10036, 6)

In [28]:
ga_first.shape

(2509, 6)

In [53]:
ga_stack.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
960,63da94b8-84ce-400e-987a-3f447ee45a0a,2023-03-31,0,0,0,1
1170,8278e572-0bfa-4bfb-a533-1696357b8cac,2023-03-16,0,0,0,1
3810,a83600c9-034a-421a-993a-14c6fbd0036e,2023-04-12,0,0,0,2
260,667e35ce-80c4-4f46-a8cf-196a3dff6c42,2023-05-05,0,0,0,1
3147,384b50a3-949f-48c7-a30a-6fefe2ee1570,2023-05-15,0,0,0,2


In [31]:
ga_stack = ga_stack[~ga_stack['due_date'].isnull()]

In [32]:
ga_stack.shape

(6429, 6)

#### Pilot Stacking

In [33]:
pilot_df.head()

,product_id,business_id,pilot_first_due_date,pilot_second_due_date,pilot_third_due_date,pilot_fpd,pilot_spd,pilot_tpd,pilot_first_cure_days,pilot_second_cure_days,pilot_third_cure_days,FPD_plus_3,SPD_plus_3,TPD_plus_3,FPD_plus_15,SPD_plus_15,TPD_plus_15,FPD_plus_30,SPD_plus_30,TPD_plus_30
0,e2feec25-251b-45a8-9f37-0304a0f30862,fdc7154e-18b3-4c26-b3b1-5c748f2facdb,2022-09-06 07:00:11.333000+00:00,2022-10-07 07:00:24.186000+00:00,2022-11-07 05:51:20.582000+00:00,0,0,1,0,0,60,0,0,1,0,0,1,0,0,1
1,28910777-199c-48d9-8a1e-a846e3040b08,35308ad5-7271-4703-a362-18a4d79cd334,2022-09-11 07:00:11.946000+00:00,2022-10-11 07:00:13.569000+00:00,2022-10-11 07:00:15.332000+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,bb977a66-20bf-4461-833b-046e67d88ef9,5cbc0f97-efb1-470a-9952-52a30b053386,2022-09-07 07:00:03.070000+00:00,2022-11-27 08:00:31.660000+00:00,2022-12-27 08:00:07.952000+00:00,1,0,0,6,0,0,1,0,0,0,0,0,0,0,0
3,eee7de77-1646-48b5-a9b6-d469d0a8a208,623d4577-8472-42c5-809b-1d72d8625b8b,2022-08-22 05:00:11.972000+00:00,2022-09-12 07:00:07.479000+00:00,2022-09-21 07:00:14.824000+00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,85a98d0f-5fb3-4dcd-9e47-435cf8555784,ab59e1a9-605b-43b4-8312-880b4123bc28,2022-10-01 07:00:44.111000+00:00,2022-10-15 07:06:47.568000+00:00,2022-10-31 07:00:16.200000+00:00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
pilot_df['pilot_first_due_date'] = pd.to_datetime(pilot_df['pilot_first_due_date']).dt.date
pilot_df['pilot_second_due_date'] = pd.to_datetime(pilot_df['pilot_second_due_date']).dt.date
pilot_df['pilot_third_due_date'] = pd.to_datetime(pilot_df['pilot_third_due_date']).dt.date

In [35]:
pilot_df.columns

Index(['product_id', 'business_id', 'pilot_first_due_date',
       'pilot_second_due_date', 'pilot_third_due_date', 'pilot_fpd',
       'pilot_spd', 'pilot_tpd', 'pilot_first_cure_days',
       'pilot_second_cure_days', 'pilot_third_cure_days', 'FPD_plus_3',
       'SPD_plus_3', 'TPD_plus_3', 'FPD_plus_15', 'SPD_plus_15', 'TPD_plus_15',
       'FPD_plus_30', 'SPD_plus_30', 'TPD_plus_30'],
      dtype='object')

In [36]:
first = pilot_df[['business_id','pilot_first_due_date','FPD_plus_3','FPD_plus_15','FPD_plus_30']].rename(columns={'pilot_first_due_date': 'due_date', 'FPD_plus_3': 'DPD_plus_3','FPD_plus_15':'DPD_plus_15','FPD_plus_30':'DPD_plus_30'})
second = pilot_df[['business_id','pilot_second_due_date','SPD_plus_3','SPD_plus_15','SPD_plus_30']].rename(columns={'pilot_second_due_date': 'due_date', 'SPD_plus_3': 'DPD_plus_3','SPD_plus_15':'DPD_plus_15','SPD_plus_30':'DPD_plus_30'})
third = pilot_df[['business_id','pilot_third_due_date','TPD_plus_3','TPD_plus_15','TPD_plus_30']].rename(columns={'pilot_third_due_date': 'due_date', 'TPD_plus_3': 'DPD_plus_3','TPD_plus_15':'DPD_plus_15','TPD_plus_30':'DPD_plus_30'})

In [37]:
first['emi_no'] = 1
second['emi_no'] = 2
third['emi_no'] = 3

In [54]:
pilot_stack = pd.concat([first, second, third], axis = 0, ignore_index = True)

In [57]:
pilot_stack.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
337,12b3f26a-767b-46b3-a49f-fa6725745a37,2022-10-09,0,0,0,1
1399,80ccf81d-a994-438f-bf19-bd8abca6e883,2022-10-21,1,1,1,3
372,1c96532d-90c4-41f0-80cd-ca66b7ad3f02,2022-08-13,0,0,0,1
345,4e8031ad-cf3e-4dc5-be6d-e9b3d0695ac4,2022-08-23,1,0,0,1
272,e280588c-cb2f-450c-985e-34291870c504,2022-09-13,0,0,0,1


In [58]:
pilot_stack.shape

(1929, 6)

#### Adjusting EMI_NO for pilot migrated accounts

In [ ]:
pilot_mg

In [60]:

ga_stack.loc[(ga_stack['business_id'].isin(list(pilot_df['business_id']))), 'emi_no'] = ga_stack.loc[(ga_stack['business_id'].isin(list(pilot_df['business_id']))), 'emi_no'] + 3

In [66]:
ga_stack.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
4245,4bcc781a-72fb-4dcd-aa4e-1ba391a423cf,2023-03-27,0,0,0,2
6503,e08e8439-93fc-48c3-8eba-6ab3df294128,2023-05-02,0,0,0,3
7365,e03a7b0f-dcd3-434b-adab-58cef9078990,2023-03-08,0,0,0,3
4553,a29f50d5-e8be-4350-8608-eddb27b5a1e3,2023-03-13,1,0,0,2
4379,c3d9ac9b-0dad-48b0-8800-61372c7c618a,2023-03-23,0,0,0,2


#### Merging Pilot and GA Stacked

In [87]:
dff = pd.concat([ga_stack, pilot_stack], axis = 0, ignore_index = True)

In [92]:
dff.shape

(8003, 6)

In [89]:
dff = dff[~(dff['business_id'].isnull())]

In [69]:
dff.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
6117,72c53eb7-6224-4e20-ac4d-550f418c3294,2023-04-28,0,0,0,7
769,06b1e823-b85b-4102-a78f-ca23ae897a07,2023-04-08,0,0,0,1
4330,e5f0a757-771a-493e-9d7b-349ddcba2991,2023-02-02,0,0,0,5
7685,a42abe38-52fc-4dbf-9fde-cda5ef443787,2022-08-28,1,0,0,2
3875,0680993b-31c7-4083-97c1-3997c44d6c72,2023-03-19,1,1,1,5


In [91]:
dff = dff[~(dff['emi_no']==7)]

In [82]:
dff[dff['business_id']== 'ddc3a22d-3205-4d6e-900a-201605bbfa9c'].sort_values('emi_no')

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
6852,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2022-10-20,0,0,0,1
7495,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2022-11-20,0,0,0,2
8138,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2022-12-20,1,0,0,3
2177,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2023-02-03,1,1,1,4
4104,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2023-03-06,0,0,0,5
5494,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2023-04-03,0,0,0,6


In [83]:
dff['emi_no'].value_counts()

1    2810
2    2231
3    1708
4     635
5     339
6     325
Name: emi_no, dtype: int64

In [117]:
# from sqlalchemy.types import NVARCHAR
# # from conf.config import SQLQuery
# q = SQLQuery('snowflake')

# dff.to_sql(name='lending_dpd_data_till_may20',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in dff})